# Machine Learning using Sentinel-2 Data

This example uses training data from the
[Coast Train](https://github.com/nick-murray/coastTrain) dataset
along with Sentinel-2 data to demonstrate how to use a
machine learning classifier, in this case, Random Forest, to
assign a class to each pixel.

This notebook combines lessons from previous notebooks into
a comprehensive worked example.

## Getting started

First we load the required Python libraries and tools.

In [ ]:
from pystac_client import Client
from dask.distributed import Client as DaskClient
from odc.stac import load, configure_s3_access
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import folium
import joblib


from sklearn.ensemble import RandomForestClassifier

import odc.geo.xr  # noqa: F401

## Study site configuration

Here we establish the STAC catalog we're using as well as a
spatial and temporal extent. This can be anywhere, but this location
near Kuching was chosen due to the training data having several
classes available.

In [ ]:
# STAC Catalog URL
# catalog = "https://stac.staging.digitalearthpacific.org"
catalog = "https://earth-search.aws.element84.com/v1"
# Create a STAC Client
client = Client.open(catalog)

<font color='blue'>1.1. Define your area of interest. Find the coordinates of the bottom left and top right corners of your bounding box / area of interest. 

Use QGIS, Google Maps or another site to find the coordinates. Make sure to use at least 4 or 5 decimal places. Lat = latitude and lon = longitude. The min is in the bottom left and max is in the top right.   

In [ ]:
# 1.1 - input your four coordinates here 
min_lat = -16.5297786
min_lon = 179.4065010
max_lat = -16.5253252
max_lon = 179.4101265

bbox = (min_lon, min_lat, max_lon, max_lat)

<font color='blue'>1.2. Define the time period of interest. 

You can now define the time you are interested in. It is goood to put the format as "year-month"/"year-month". For example "2022-06/2024-09" to get all images covering June 2022 to September 2024. You can try to just choose the flowering months in one year but may have issues with cloud cover. Try different time periods and see what happes. 

In [ ]:
# 1.2 - input your datetime here - recommend at least 3 months and max 3 years. 
datetime = "2024-01/2024-12"

In [ ]:
# Create local dask cluster to improve data load time. Only run this once.
dask_client = DaskClient(n_workers=1, threads_per_worker=16, memory_limit='16GB')

# Configure S3 access. Cloud defaults is an optimisation, while requester pays is required for Landsat
configure_s3_access(cloud_defaults=True, requester_pays=True)

## Training data

Next up we gather training data. This could be any geospatial point dataset
with a column that is numeric, for the class.

If you'd like to explore the structure of this data, you can run `gdf.head()`
to see the first few rows. The `explore()` function with the `column` argument
will show the data on the map, and change the colour based on that column.

<font color='blue'>2.1. Input your data from the field.  

If you have new data, save your data as a geojson in QGIS and then drag and drop it into the same folder as this notebook in DEP. Then you will have to put the name of the file in the brackets below inside of quotes: eg. `'name.geojson'`   

In [ ]:
# 2.1 - input your data file here inside the '' 
gdf = gpd.read_file('Training_Data/Korotari_clean1.geojson', bbox=bbox)

# gdf = gdf.fillna(0)
gdf.explore(column="Random_Forest", legend=True)

## Find and load Sentinel-2 data

Here we search for Sentinel-2 scenes over our study area and use
Dask to lazy-load them. We're only loading the red, green, blue, nir and swir
bands, along with the scene classification (scl) band.

<font color='blue'>2.2. - define the satellite image collections you wish to use: 

Input your collections name inside the brackets: 

You may try:   
"sentinel-2-c1-l2a" (10mx10m resolution pixels)   
OR   
"landsat-c2-l2" (30mx30m resolution pixels)

<font color='blue'>2.3. - define your cloud cover threshold. We recommend a number somewhere between 10-50. No need for quotes. 

Write this next to "lt": eg. {"lt": 50}} 


In [ ]:
# Search for Sentinel-2 data
items = client.search(
    collections=["sentinel-2-c1-l2a"],
    bbox=bbox,
    datetime=datetime,
    query={"eo:cloud_cover": {"lt": 40 }},
).item_collection()

print(f"Found {len(items)} items")

In [ ]:
# Load the data into an xarray Dataset
data = load(
    items,
    measurements=["red", "green", "blue", "nir08", "swir16", "scl"],
    bbox=bbox,
    chunks={"x": 2048, "y": 2048},
    groupby="solar_day",
)

data

## Data preparation

Now that we have data, we need to clean it up, masking out clouds
and scaling values to between 0-1, which are the valid reflectance
values.

We add a couple of indices too, which will help the machine learning
algorithm.

Note that we still have a lazy-loaded array, and haven't transferred
any data over the network.

In [ ]:
# Mask out clouds and scale values

# Apply Sentinel-2 cloud mask
# 1: defective, 3: shadow, 9: high confidence cloud, 10: thin cirrus
mask_flags = [1, 3, 9, 10]

cloud_mask = ~data.scl.isin(mask_flags)
masked = data.where(cloud_mask)

# Apply scaling and clip to valid data, from 0 to 1
scaled = (masked.where(masked != 0) * 0.0001).clip(0, 1)

# Add some indices
scaled["ndvi"] = (scaled.nir08 - scaled.red) / (scaled.nir08 + scaled.red)
# scaled["ndwi"] = (scaled.green - scaled.nir08) / (scaled.green + scaled.nir08)

scaled


In [ ]:
# Visualise one date, to make sure it looks good.
# This example shows empty areas where we've masked out nodata, but
# note that there are still a lot of clouds coming in!

scaled.isel(time=0).odc.explore(vmin=0, vmax=0.3)

## Create a cloud-free composite

The final data preparation step involves creating a temporal
median of the data bands. Here we use `compute()` to process
the data and bring it into memory.

We preview the data in the second cell below.

In [ ]:
# Create a median composite, which should get rid of most of the remaining clouds
# Note that this will take a few minutes to complete

median = scaled.median("time").compute()

median

<font color='blue'>3.1. - visualise the resulting median satellite image:

`median.odc.explore(vmin=0, vmax=0.3)`

In [ ]:
median.odc.explore(vmin=0, vmax=0.3)

## Prepare training data array

This next step involves extracting observed values from the satellite data
and combining them with our point data, resulting in something like this:

`class, red, green, blue ...`

This structure is then fed into the machine learning classifier.

In [ ]:
# First transform the training points to the same CRS as the data
training = gdf.to_crs(median.odc.geobox.crs)

# Next get the X and Y values out of the point geometries
training_da = training.assign(x=training.geometry.x, y=training.geometry.y).to_xarray()

# Now we can use the x and y values (lon, lat) to extract values from the median composite
training_values = (
    median.sel(training_da[["x", "y"]], method="nearest").squeeze().compute().to_pandas()
)

# Join the training data with the extracted values and remove unnecessary columns
training_array = pd.concat([training["Random_Forest"], training_values], axis=1)
training_array = training_array.drop(
    columns=[
        "y",
        "x",
        "spatial_ref",
    ]
)

# Drop rows where there was no data available
training_array = training_array.dropna()

# Preview our resulting training array
training_array.head()

## Create a classifier and fit a model

We pass in simple numpy arrays to the classifier, one has the
observations (the values of the red, green, blue and so on)
while the other has the classes.

In [ ]:
# The classes are the first column
classes = np.array(training_array)[:, 0]

# The observation data is everything after the first column
observations = np.array(training_array)[:, 1:]

# Create a model...
classifier = RandomForestClassifier()

# ...and fit it to the data
model = classifier.fit(observations, classes)

## Prediction

Next we predict. Again, we need a simple numpy array, this time
just with the observations. This needs to be in long array where
the x dimension is the observation values and the y is each cell
in the original raster.

In [ ]:
# Convert to a stacked array of observations
stacked_arrays = median.to_array().stack(dims=["y", "x"]).transpose()

# Predict the classes
predicted = model.predict(stacked_arrays)

# Reshape back to the original 2D array
array = predicted.reshape(len(median.y), len(median.x))

# Convert to an xarray again, because it's easier to work with
predicted_da = xr.DataArray(
    array, coords={"y": masked.y, "x": masked.x}, dims=["y", "x"]
)

## Visualise our results

Here we're visualising the results along with the RGB image
and the original training data points. We're doing this using
a Python library called Folium.

In [ ]:
print(predicted_da.dtype)  # Check the dtype of your DataArray
predicted_da = predicted_da.astype('float32')  # Convert to float32

In [ ]:
# Put it all on a single interactive map
# center = [np.mean([min_lat[0], max_lat[0]]), np.mean([min_lat[1], max_lat[1]])]
# m = folium.Map(location=center, zoom_start=11)

center = [(min_lat + max_lat) / 2, (min_lon + max_lon) / 2]  # Assuming min_lon and max_lon are defined
m = folium.Map(location=center, zoom_start=11)



# RGB for the median
median.odc.to_rgba(vmin=0, vmax=0.3).odc.add_to(m, name="Median Composite")



<font color='blue'>4.1. - visualise the resulting machine learning prediction:

The name of your model prediction is `predicted_da` so input this before `odc.add_to`

In [ ]:
# Categorical for the predicted classes and for the training data
predicted_da.odc.add_to(m, name="Predicted")
gdf.explore(m=m, column="Random_Forest", legend=True, name="Training Data")

# Layer control
folium.LayerControl().add_to(m)

m

In [ ]:
# use the trained sklearn model (in your code earlier `model = classifier.fit(...)`)
# ensure 'model' exists and observations/classes are defined as numpy arrays

# If observations may contain NaNs, filter them out first:
import numpy as np
mask_valid = ~np.any(np.isnan(observations), axis=1)   # True for rows without NaN
print("Valid training rows:", mask_valid.sum(), "out of", len(mask_valid))

train_preds = np.full(len(observations), np.nan)
if mask_valid.sum() > 0:
    train_preds[mask_valid] = model.predict(observations[mask_valid])

# Report using only valid rows
from sklearn.metrics import classification_report, confusion_matrix
valid_idx = mask_valid
print(classification_report(classes[valid_idx], train_preds[valid_idx]))
print("Confusion matrix:\n", confusion_matrix(classes[valid_idx], train_preds[valid_idx]))

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

# ------------------------------------------------------------------
# 1. Filter out rows with NaNs (same as before)
# ------------------------------------------------------------------
mask_valid = ~np.any(np.isnan(observations), axis=1)   # True for rows without NaN
print("Valid rows (no NaN):", mask_valid.sum(), "out of", len(mask_valid))

X_valid = observations[mask_valid]
y_valid = classes[mask_valid]

# ------------------------------------------------------------------
# 2. Remove classes that have only 1 sample (needed for stratify)
# ------------------------------------------------------------------
unique_classes, counts = np.unique(y_valid, return_counts=True)
rare_classes = unique_classes[counts < 2]

if len(rare_classes) > 0:
    print("Removing rare classes with only 1 sample:", rare_classes)

    mask_enough = ~np.isin(y_valid, rare_classes)
    X_valid = X_valid[mask_enough]
    y_valid = y_valid[mask_enough]

print("Samples remaining after removing rare classes:", len(y_valid))

# ------------------------------------------------------------------
# 3. 80/20 train–test split (stratified)
# ------------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_valid,
    y_valid,
    test_size=0.2,       # 20% test
    random_state=42,     # for reproducibility
    stratify=y_valid     # keep class proportions
)

print("Train size:", len(y_train), "Test size:", len(y_test))

# ------------------------------------------------------------------
# 4. Train the model on the training set
#    (assumes you already defined `classifier`, e.g. RandomForestClassifier(...))
# ------------------------------------------------------------------
model = classifier.fit(X_train, y_train)

# ------------------------------------------------------------------
# 5. Predict on the test set
# ------------------------------------------------------------------
y_pred = model.predict(X_test)

# ------------------------------------------------------------------
# 6. Metrics: classification report, confusion matrix,
#    plus explicit precision / recall / F1
# ------------------------------------------------------------------
print("Classification report (80/20 split):")
print(classification_report(y_test, y_pred))

print("Confusion matrix:")
print(confusion_matrix(y_test, y_pred))

# Overall scores (change 'weighted' to 'macro' if you prefer)
precision = precision_score(y_test, y_pred, average="weighted", zero_division=0)
recall    = recall_score(y_test, y_pred, average="weighted", zero_division=0)
f1        = f1_score(y_test, y_pred, average="weighted", zero_division=0)

print(f"Weighted precision: {precision:.3f}")
print(f"Weighted recall:    {recall:.3f}")
print(f"Weighted F1-score:  {f1:.3f}")



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

# Recompute just to keep it self-contained
cm = confusion_matrix(y_test, y_pred)

# Match these labels to your actual class codes (sorted by row/column order)
class_labels = np.unique(y_test)

fig, ax = plt.subplots(figsize=(6, 6))
im = ax.imshow(cm, interpolation="nearest")
ax.set_title("Confusion Matrix (80/20 split)")
fig.colorbar(im)

ax.set_xticks(np.arange(len(class_labels)))
ax.set_yticks(np.arange(len(class_labels)))
ax.set_xticklabels(class_labels)
ax.set_yticklabels(class_labels)

plt.setp(ax.get_xticklabels(), rotation=45, ha="right")

ax.set_xlabel("Predicted label")
ax.set_ylabel("True label")

# Optional: add counts on each cell
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, cm[i, j],
                ha="center", va="center")

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)
import seaborn as sns

# ------------------------------------------------------------------
# 1️⃣ Filter out NaN rows
# ------------------------------------------------------------------
mask_valid = ~np.any(np.isnan(observations), axis=1)
X_valid = observations[mask_valid]
y_valid = classes[mask_valid]
print("Valid samples:", len(y_valid))

# ------------------------------------------------------------------
# 2️⃣ Remove any classes with only 1 sample (stratify requirement)
# ------------------------------------------------------------------
unique_classes, counts = np.unique(y_valid, return_counts=True)
rare_classes = unique_classes[counts < 2]
if len(rare_classes) > 0:
    print("Removing rare classes:", rare_classes)
    mask_enough = ~np.isin(y_valid, rare_classes)
    X_valid = X_valid[mask_enough]
    y_valid = y_valid[mask_enough]

print("Samples used:", len(y_valid))

# ------------------------------------------------------------------
# 3️⃣ Train/Test split 80/20
# ------------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_valid,
    y_valid,
    test_size=0.2,
    random_state=42,
    stratify=y_valid
)

print("Train:", len(y_train), "Test:", len(y_test))

# ------------------------------------------------------------------
# 4️⃣ Fit + Predict
# ------------------------------------------------------------------
model = classifier.fit(X_train, y_train)
y_pred = model.predict(X_test)

# ------------------------------------------------------------------
# 5️⃣ Print Classification Report + Precision / Recall / F1
# ------------------------------------------------------------------
print("=== Classification Report (80/20 Split) ===")
print(classification_report(y_test, y_pred))

precision = precision_score(y_test, y_pred, average="weighted", zero_division=0)
recall    = recall_score(y_test, y_pred, average="weighted", zero_division=0)
f1        = f1_score(y_test, y_pred, average="weighted", zero_division=0)

print(f"Weighted Precision: {precision:.3f}")
print(f"Weighted Recall:    {recall:.3f}")
print(f"Weighted F1-score:  {f1:.3f}")

# ------------------------------------------------------------------
# 6️⃣ Confusion Matrix — NICE visual
# ------------------------------------------------------------------
cm = confusion_matrix(y_test, y_pred)
labels = np.unique(y_test)

plt.figure(figsize=(8, 6))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=labels,
    yticklabels=labels,
    cbar_kws={"shrink": 0.8}
)

plt.title("Confusion Matrix (80/20 Split)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create normalized percentage confusion matrix
cm_norm = cm / cm.sum(axis=1, keepdims=True) * 100

plt.rcParams.update({
    "font.size": 14,
    "axes.titlesize": 16,
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "figure.dpi": 300  # high resolution
})

fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# ---------- COUNTS HEATMAP ----------
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Spectral_r",
    linewidths=0.7,
    linecolor="black",
    xticklabels=labels,
    yticklabels=labels,
    ax=axes[0],
    cbar=False
)
axes[0].set_title("Confusion Matrix (Counts)", fontweight="bold")
axes[0].set_xlabel("Predicted Label", fontweight="bold")
axes[0].set_ylabel("True Label", fontweight="bold")

# ---------- NORMALIZED PERCENT HEATMAP ----------
sns.heatmap(
    cm_norm,
    annot=True,
    fmt=".1f",
    cmap="turbo",
    linewidths=0.7,
    linecolor="black",
    xticklabels=labels,
    yticklabels=labels,
    ax=axes[1],
    cbar=True,
    cbar_kws={"shrink": 0.7, "label": "% of True Class"}
)
axes[1].set_title("Confusion Matrix (%)", fontweight="bold")
axes[1].set_xlabel("Predicted Label", fontweight="bold")
axes[1].set_ylabel("")

plt.suptitle("Model Performance – 80/20 Split", fontsize=18, fontweight="bold")
plt.tight_layout()

# Save for paper (PNG high-res)
plt.savefig("confusion_matrix_paper_ready.png",
            dpi=400, bbox_inches="tight", transparent=False)

plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

# Get classification report as a dictionary
report = classification_report(y_test, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()

# Remove summary rows if you want only class metrics
df_classes = df_report.drop(["accuracy", "macro avg", "weighted avg"], errors="ignore")

# Only keep useful columns
df_classes = df_classes[["precision", "recall", "f1-score", "support"]]

# Create heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(
    df_classes,
    annot=True,
    fmt=".2f",
    cmap="viridis",   # <-- change colormap here if desired
    linewidths=0.7,
    linecolor="black",
    cbar=True
)

plt.title("Classification Report (Color-coded Table)", fontsize=14, fontweight="bold")
plt.xlabel("Metrics", fontweight="bold")
plt.ylabel("Class Label", fontweight="bold")
plt.tight_layout()

# Save high resolution version for paper use
plt.savefig("classification_report_colortable.png", dpi=400, bbox_inches="tight")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Your class name mapping
class_names = {
    1.0: "African_Tulip",
    3.0: "Breadfruit_Tree",
    4.0: "Cocoa_Tree",
    5.0: "Cordia_Salmwood",
    7.0: "Guava_Tree",
    8.0: "Merremia_Peltata",
    9.0: "Native_Forest",
    10.0: "Raintree",
    13.0: "Shrubs"
}

# Convert labels into species names in the same order as cm rows/columns
label_names = [class_names[l] for l in labels]

# Create normalized percentage confusion matrix
cm_norm = cm / cm.sum(axis=1, keepdims=True) * 100

plt.rcParams.update({
    "font.size": 14,
    "axes.titlesize": 16,
    "axes.labelsize": 14,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "figure.dpi": 300
})

fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# ---------- COUNTS HEATMAP ----------
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Spectral_r",
    linewidths=0.7,
    linecolor="black",
    xticklabels=label_names,
    yticklabels=label_names,
    ax=axes[0],
    cbar=False
)
axes[0].set_title("Confusion Matrix (Counts)", fontweight="bold")
axes[0].set_xlabel("Predicted Species", fontweight="bold")
axes[0].set_ylabel("True Species", fontweight="bold")
axes[0].tick_params(axis='x', rotation=45)

# ---------- NORMALIZED PERCENT HEATMAP ----------
sns.heatmap(
    cm_norm,
    annot=True,
    fmt=".1f",
    cmap="turbo",
    linewidths=0.7,
    linecolor="black",
    xticklabels=label_names,
    yticklabels=label_names,
    ax=axes[1],
    cbar=True,
    cbar_kws={"shrink": 0.7, "label": "% of True Class"}
)
axes[1].set_title("Confusion Matrix (%)", fontweight="bold")
axes[1].set_xlabel("Predicted Species", fontweight="bold")
axes[1].set_ylabel("")
axes[1].tick_params(axis='x', rotation=45)

plt.suptitle("Model Performance – 80/20 Split (Species-Level Confusion Matrix)", fontsize=18, fontweight="bold")
plt.tight_layout()

plt.savefig("confusion_matrix_with_species.png", dpi=400, bbox_inches="tight", transparent=False)
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

# --- Your mapping of class numbers to species names ---
class_names = {
    1.0: "African_Tulip",
    3.0: "Breadfruit_Tree",
    4.0: "Cocoa_Tree",
    5.0: "Cordia_Salmwood",
    7.0: "Guava_Tree",
    8.0: "Merremia_Peltata",
    9.0: "Native_Forest",
    10.0: "Raintree",
    13.0: "Shrubs"
}

# --- Step 1: Get classification report dictionary ---
report = classification_report(y_test, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()

# --- Step 2: Remove summary rows (accuracy, macro avg, weighted avg) ---
df_classes = df_report.drop(["accuracy", "macro avg", "weighted avg"], errors="ignore")

# --- Step 3: Replace numeric class labels with species names ---
df_classes.index = [
    class_names[float(i)] if i not in ["accuracy", "macro avg", "weighted avg"] else i
    for i in df_classes.index
]

# --- Step 4: Keep essential metrics ---
df_classes = df_classes[["precision", "recall", "f1-score", "support"]]

# --- Step 5: Create heatmap ---
plt.figure(figsize=(12, 7))
sns.heatmap(
    df_classes,
    annot=True,
    fmt=".2f",
    cmap="viridis",
    linewidths=0.7,
    linecolor="black",
    cbar=True
)

plt.title("Classification Report (Species-Level, Color-coded Table)", fontsize=16, fontweight="bold")
plt.xlabel("Metrics", fontweight="bold")
plt.ylabel("Species", fontweight="bold")
plt.xticks(rotation=0)
plt.tight_layout()

# --- Step 6: Save high-res figure ---
plt.savefig("classification_report_species_colortable.png", dpi=400, bbox_inches="tight")
plt.show()


<font color='blue'>4.2. - write this model output map to a new file:

Input the name of your model prediction before `.odc.write_cog`

In [ ]:
predicted_da.odc.write_cog("Korotari_v2.tif", overwrite=True)
# predicted_da.plot.imshow()

In [ ]:
# Check the data type of the 'Random_Forest' column
print(gdf['Random_Forest'].dtype)

# Display the first few rows of the 'Random_Forest' column to inspect its contents
print(gdf['Random_Forest'].head())

In [ ]:
# Assuming the column contains text data
african_tulip_count = gdf[gdf['Random_Forest'] == '6'].shape[0]

In [ ]:
# Assuming '2' represents African Tulip in the Random_Forest column

# Total number of predictions
total_predictions = gdf['Random_Forest'].count()

# Number of African Tulip predictions (where Random_Forest equals 2)
african_tulip_count = gdf[gdf['Random_Forest'] == 2].shape[0]

# Calculate the percentage of African Tulip
percentage_african_tulip = (african_tulip_count / total_predictions) * 100

# Print the result
print(f"Percentage of Settlements: {percentage_african_tulip:.2f}%")


In [ ]:
rf_percentage = gdf["Random_Forest"].value_counts(normalize=True) * 100
print(rf_percentage)

<font color='blue'>5.1. - download the file, load, explore and set new colours in QGIS:


In [ ]:
joblib.dump(model, "cordia_v.2.0.model")

## Considerations

Do the results make sense?

What are some of the limitations of the visualisation?

### Next steps

The obvious next step is to fine tune the data. Perhaps download the points for this
region of interest as well as the RGB image and add and remove points until
there is a more representative training dataset.

### New AOI for your interest

Choose a new AoI and ToI based on your interests. This could be for wider Tongatapu, Eua or even Vava'u.  

In [ ]:
min_lat = -17.772398
min_lon = 178.401739
max_lat = -17.766376
max_lon = 178.407854

bbox = [min_lon, min_lat, max_lon, max_lat]

In [ ]:
# 1.1 - input your four coordinates here 
min_lat = -17.772421
min_lon = 178.37
max_lat = -17.766539
max_lon = 178.401809

bbox_wainibuka = [min_lon, min_lat, max_lon, max_lat]

In [ ]:
# Search for Sentinel-2 data
items = client.search(
    collections=["sentinel-2-c1-l2a"],
    bbox=bbox_wainibuka,
    datetime=datetime,
    query={"eo:cloud_cover": {"lt": 15 }},
).item_collection()

print(f"Found {len(items)} items")

In [ ]:
# Load the data into an xarray Dataset
data_Wainibuka = load(
    items,
    measurements=["red", "green", "blue", "nir08", "swir16", "scl"],
    bbox=bbox_wainibuka,
    chunks={"x": 800, "y": 800},
    groupby="solar_day",
)

# data_Eua

In [ ]:
# Mask out clouds and scale values

# Apply Sentinel-2 cloud mask
# 1: defective, 3: shadow, 9: high confidence cloud, 10: thin cirrus
mask_flags = [1, 3, 9, 10]

cloud_mask = ~data_Wainibuka.scl.isin(mask_flags)
masked = data_Wainibuka.where(cloud_mask)

# Apply scaling and clip to valid data, from 0 to 1
scaled_Wainibuka = (masked.where(masked != 0) * 0.0001).clip(0, 1)

# Add some indices
scaled_Wainibuka["ndvi"] = (scaled_Wainibuka.nir08 - scaled_Wainibuka.red) / (scaled_Wainibuka.nir08 + scaled_Wainibuka.red)
# scaled["ndwi"] = (scaled.green - scaled.nir08) / (scaled.green + scaled.nir08)

# Create a median composite, which should get rid of most of the remaining clouds
# Note that this will take a few minutes to complete

median_Wainibuka = scaled_Wainibuka.median("time").compute()

# median

median_Wainibuka.odc.explore(vmin=0, vmax=0.3)

In [ ]:
# Convert to a stacked array of observations
stacked_arrays = median_Wainibuka.to_array().stack(dims=["y", "x"]).transpose()

# Predict the classes
predicted_Wainibuka = model.predict(stacked_arrays)

# Reshape back to the original 2D array
array = predicted_Wainibuka.reshape(len(median_Wainibuka.y), len(median_Wainibuka.x))

# Convert to an xarray again, because it's easier to work with
predicted_da = xr.DataArray(
    array, coords={"y": masked.y, "x": masked.x}, dims=["y", "x"]
)

In [ ]:
print(predicted_da.dtype)  # Check the dtype of your DataArray
predicted_da = predicted_da.astype('float32')  # Convert to float32

# Put it all on a single interactive map
# center = [np.mean([min_lat[0], max_lat[0]]), np.mean([min_lat[1], max_lat[1]])]
# m = folium.Map(location=center, zoom_start=11)

center = [(min_lat + max_lat) / 2, (min_lon + max_lon) / 2]  # Assuming min_lon and max_lon are defined
m = folium.Map(location=center, zoom_start=11)

# RGB for the median
median_Wainibuka.odc.to_rgba(vmin=0, vmax=0.3).odc.add_to(m, name="Median Composite")


In [ ]:
# Categorical for the predicted classes and for the training data
predicted_da.odc.add_to(m, name="Predicted")
gdf.explore(m=m, column="Random_Forest", legend=True, name="Training Data")

# Layer control
folium.LayerControl().add_to(m)

m

In [ ]:
predicted_da.odc.write_cog("AT_Wainibuka_Jan_Oct_2024.tif", overwrite=True)
# predicted_da.plot.imshow()